<a href="https://colab.research.google.com/github/mzucali/didactics/blob/main/PCTO_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- SETUP COLAB ---
import sys, subprocess, importlib, math
def pip_install(p):
    try: importlib.import_module(p.split("==")[0])
    except ImportError: subprocess.check_call([sys.executable, "-m", "pip", "install", p])

for p in ["numpy","pandas","matplotlib","mplstereonet","scikit-learn","scipy"]:
    pip_install(p)

import numpy as np, pandas as pd, matplotlib.pyplot as plt
import mplstereonet
from sklearn.cluster import KMeans
from numpy.linalg import svd, eig


STEREONET and PLOT

In [ ]:
def pole_from_strike_dip(strike_deg, dip_deg):
    # ritorna trend/plunge del polo (right-hand rule, gradi)
    strike = np.deg2rad(strike_deg)
    dip = np.deg2rad(dip_deg)
    # normale al piano in ENU
    n = np.array([-np.sin(dip)*np.sin(strike),
                  np.sin(dip)*np.cos(strike),
                  np.cos(dip)])
    # trend/plunge
    trend = (np.rad2deg(np.arctan2(n[0], n[1]))+360)%360
    plunge = np.rad2deg(np.arcsin(n[2]))
    return trend, plunge

def cart_from_trend_plunge(trend_deg, plunge_deg):
    T, P = np.deg2rad(trend_deg), np.deg2rad(plunge_deg)
    x = np.sin(T)*np.cos(P); y = np.cos(T)*np.cos(P); z = np.sin(P)
    v = np.array([x,y,z]); return v/np.linalg.norm(v)


FISHER STATS

In [ ]:
def fisher_mean_alpha95(vectors):
    R = np.sum(vectors, axis=0); R_len = np.linalg.norm(R)
    kappa = (R_len*(vectors.shape[0]-1))/(vectors.shape[0]-R_len+1e-9)
    alpha95 = np.rad2deg(np.arccos(1 - (vectors.shape[0]-R_len)/R_len * (20**(1/(vectors.shape[0]-1))-1)))
    mean_dir = R/R_len
    return mean_dir, kappa, alpha95


KMEANS

In [ ]:
def spherical_kmeans(U, k, n_init=10):
    # U: (n,3) unit vectors
    best=None; best_inertia=np.inf
    for _ in range(n_init):
        km = KMeans(n_clusters=k, n_init=1).fit(U)
        C = km.cluster_centers_
        C = C/np.linalg.norm(C,axis=1,keepdims=True)  # proietta su sfera
        labels = np.argmin(((U[:,None,:]-C[None,:,:])**2).sum(2), axis=1)
        inertia = np.mean([np.arccos(np.clip(np.dot(U[i],C[labels[i]]),-1,1))**2 for i in range(len(U))])
        if inertia<best_inertia: best=(C,labels), best_inertia=inertia
    return best[0]  # (centers_unit, labels)


STRESS INVERSION

In [ ]:
def stress_inversion_faults(strike, dip, rake):
    # converte piani+slip in tensor minimo quadrati (soluzione didattica compatta)
    # 1) direzioni: normale n, direzione slip s (unit)
    def n_from_sd(s,d):
        s, d = np.deg2rad(s), np.deg2rad(d)
        return np.array([-np.sin(d)*np.sin(s), np.sin(d)*np.cos(s), np.cos(d)])
    def slip_dir(s,d,r):
        # rake su piano: costruisci basi sul piano e combina
        s, d, r = map(np.deg2rad,[s,d,r])
        strike_vec = np.array([ np.sin(s), np.cos(s), 0.0])
        dip_vec    = np.array([ np.cos(s)*np.cos(d), -np.sin(s)*np.cos(d), np.sin(d)])
        return (np.cos(r)*strike_vec + np.sin(r)*dip_vec)

    N=[]; S=[]
    for S0,D0,R0 in zip(strike,dip,rake):
        n = n_from_sd(S0,D0); s = slip_dir(S0,D0,R0)
        N.append(n/np.linalg.norm(n)); S.append(s/np.linalg.norm(s))
    N, S = np.array(N), np.array(S)

    # 2) minimizza ||(σ·n)_tan - s||: vincola Tr(σ)=1 (didattico)
    # usa soluzione SVD su design matrix (semplificata)
    A=[]; b=[]
    for n,s in zip(N,S):
        P = np.eye(3)-np.outer(n,n)     # proiettore tangenziale
        # (σ·n)_tan = P·σ·n  ~ s
        # vettorizza σ simmetrico: [σxx, σyy, σzz, σxy, σxz, σyz]
        v = np.array([
            P[0,0]*n[0], P[1,1]*n[1], P[2,2]*n[2],
            P[0,1]*n[1]+P[1,0]*n[0],
            P[0,2]*n[2]+P[2,0]*n[0],
            P[1,2]*n[2]+P[2,1]*n[1],
        ])
        A.append(np.vstack([v,])*3)     # 3 eq per x,y,z (approccio compatto)
        b.append(s)
    A = np.vstack(A); b = np.concatenate(b)
    # risolvi LS
    x, *_ = np.linalg.lstsq(A, b, rcond=None)
    Sxx,Syy,Szz,Sxy,Sxz,Syz = x
    Sigma = np.array([[Sxx,Sxy,Sxz],[Sxy,Syy,Syz],[Sxz,Syz,Szz]])
    # autovalori/vettori → assi principali
    w,V = eig((Sigma+Sigma.T)/2)
    order = np.argsort(w)[::-1]  # σ1≥σ2≥σ3
    return w[order].real, V[:,order].real  # (eigenvalues, principal axes)


Output didattici:

stereonet con poli/linee, cluster colorati, α95;

tabella assi σ1–σ3 (trend/plunge) e rapporto φ;

suggerimenti su qualità dati (misfit medio, outlier).